In [ ]:
# Reference  tutorial 
# amber https://jerkwin.github.io/2018/01/07/Amber%E5%9F%BA%E7%A1%80%E6%95%99%E7%A8%8BB5-%E6%A8%A1%E6%8B%9F%E7%BB%BF%E8%89%B2%E8%8D%A7%E5%85%89%E8%9B%8B%E7%99%BD%E5%8F%8A%E6%9E%84%E5%BB%BA%E4%BF%AE%E9%A5%B0%E7%9A%84%E6%B0%A8%E5%9F%BA%E9%85%B8%E6%AE%8B%E5%9F%BA/
# http://ambermd.org/tutorials/basic/tutorial5/
# https://docs.bioexcel.eu/2020_06_09_online_ambertools4cp2k/03-prepPDB/index.html

In [ ]:
#manually  modify the pdb file of 6oim， spilt the pdb to 3 fragments
# in order to perpare the force field for this non standard protein
# 1st-12th amino acid 
# 13th non-standard amino acid
# 14th-end

In [ ]:

#delete the hydrogen atom for the fragment amino acid sequence
!pdb4amber -i ./data/6oim_CYX12_sotorasib_1_12.pdb -o ./data/6oim_CYX12_sotorasib_1_12_delH.pdb --nohyd --dry
!pdb4amber -i ./data/6oim_CYX12_sotorasib_14_e.pdb -o ./data/6oim_CYX12_sotorasib_14_e_delH.pdb --nohyd --dry


In [ ]:
# generate the pdb file for non-standard amino acid binding with sotorasib
# this step need the assistance from the professor drug chemistry
# put the sequence together
!cat ./data/6oim_CYX12_sotorasib_1_12_delH.pdb  ./data/CYX.pdb  ./data/6oim_CYX12_sotorasib_14_e_delH.pdb > ./data/6oim_CYX12_sotorasib_clean.pdb

In [73]:
# generate .ac file. Notice : In mol2, sdf, cif files, the atom types and order must be exactly the same as in pdb files, otherwise the generated files will be unusable.
# -at must use amber, files generated with other parameters are not usable, such as gaff2.
!antechamber -i ./data/CYX12_sotorasib.mol2 -fi mol2 -o ./data/sotor_amber.ac -fo ac -c bcc -nc 0 -rn CYX -at amber


Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to amber; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: Total number of electrons: 358; net charge: 0

Running: /opt/homebrew/Caskroom/miniconda/base/envs/openmm/bin/sqm -O -i sqm.in -o sqm.out



In [97]:
#Generate a prepin file using an ac file. 
#Edit the .mc file manually according the condition of  non-standard amino acid.
#Note: Manually change the first NT to N, this is very important!!
!prepgen -i ./data/sotor_amber.ac -o ./data/sotor_amber.prepin -m ./data/6oim_CYX.mc -rn CYX


PRE_HEAD_TYPE is     C
POST_TAIL_TYPE is     N
Net charge of truncated molecule is     0.00
HEAD_ATOM      1    N
TAIL_ATOM      8    C
MAIN_CHAIN     1    1    N
MAIN_CHAIN     2    2   CA
MAIN_CHAIN     3    8    C
OMIT_ATOM      1   83  O83
OMIT_ATOM      2   84  H84
OMIT_ATOM      3   85  H85
Number of mainchain atoms (including head and tail atom):     3
Number of omited atoms:     3
         change "M" to "-M" if ATOM N23 is linked to the immediate previous residue
         change "M" to "+M" if ATOM N23 is linked to the immediate after residue

In [98]:
#Use parmchk2 to generate force field files for non-standard amino acid, based on the prepin file.
#Manually check the generated file to see if there are too many ATTN.
!parmchk2 -i ./data/sotor_amber.prepin -f prepi -o ./data/sotor_amber.frcmod -a Y -s gaff2

In [ ]:
#Preparation of GDP-related files: parameters and force field files, it has a -3 charge

!antechamber -i ./data/6oim_GDP.sdf -fi sdf -o ./data/gdp_amber.mol2 -fo mol2 -c bcc -nc -3 -rn GDP -at gaff2
#calculate the force field files
!parmchk2 -i gdp_amber.mol2 -f mol2 -o ./data/gdp_amber.frcmod -s gaff2

In [101]:
#Using the .in file and the tleap tool, assemble the pdb fragments, GDP, and Mg ions together
#Prepare the molecular dynamics system, adding solvent and ions
!tleap -s -f ./data/6oim_sotor_amber.in

-I: Adding /opt/homebrew/Caskroom/miniconda/base/envs/openmm/dat/leap/prep to search path.
-I: Adding /opt/homebrew/Caskroom/miniconda/base/envs/openmm/dat/leap/lib to search path.
-I: Adding /opt/homebrew/Caskroom/miniconda/base/envs/openmm/dat/leap/parm to search path.
-I: Adding /opt/homebrew/Caskroom/miniconda/base/envs/openmm/dat/leap/cmd to search path.
-s: Ignoring all leaprc startup files.
-f: Source ./data/6oim_sotor_amber.in.

Welcome to LEaP!
Sourcing: ./data/6oim_sotor_amber.in
----- Source: /opt/homebrew/Caskroom/miniconda/base/envs/openmm/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /opt/homebrew/Caskroom/miniconda/base/envs/openmm/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /opt/homebrew/Caskroom/miniconda/base/envs/openmm/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /opt/homebrew/Caskroom/miniconda/base/envs/openmm/dat/leap/parm/frcmod.ff14SB
Reading force fi

In [78]:
from openmm import app
import openmm as mm
from openmm import unit 
from tqdm import tqdm

In [92]:
prmtop = app.AmberPrmtopFile('./data/6oim_CYX_amber_success.prmtop')
inpcrd = app.AmberInpcrdFile('./data/6oim_CYX_amber_success.inpcrd')
system = prmtop.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1*unit.nanometer,constraints=app.HBonds)